In [ ]:
# 安装 Earth Engine 和 geemap（可视化辅助）
!pip install earthengine-api
!pip install geemap




In [ ]:
!pip install ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.1 MB/s eta 0:00:00


In [ ]:
# 初始化
import ee
import geemap
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
# 认证和初始化 Earth Engine
ee.Authenticate()
ee.Initialize(project='lake-462714')



In [ ]:

# 湖泊列表
lakes = [
    {"name": "Lake Victoria", "lat": -1.0, "lon": 33.0, "continent": "Africa"},
    {"name": "Lake Tanganyika", "lat": -6.3, "lon": 29.8, "continent": "Africa"},
    {"name": "Lake Chad", "lat": 13.0, "lon": 14.0, "continent": "Africa"},
    {"name": "Lake Tana", "lat": 12.0, "lon": 37.3, "continent": "Africa"},
    {"name": "Lake Albert", "lat": 1.7, "lon": 30.9, "continent": "Africa"},
    {"name": "Lake Malawi", "lat": -12.0, "lon": 34.6, "continent": "Africa"},
    {"name": "Lake Turkana", "lat": 3.5, "lon": 36.0, "continent": "Africa"},
    {"name": "Lake Eyasi", "lat": -3.6, "lon": 35.0, "continent": "Africa"},
    {"name": "Lake Volta", "lat": 7.5, "lon": -0.1, "continent": "Africa"},
    {"name": "Lake Mweru", "lat": -9.3, "lon": 28.8, "continent": "Africa"},
    {"name": "Great Salt Lake", "lat": 41.2, "lon": -112.6, "continent": "North America"},
    {"name": "Lake Ontario", "lat": 43.5, "lon": -77.5, "continent": "North America"},
    {"name": "Lake Erie", "lat": 42.2, "lon": -81.2, "continent": "North America"},
    {"name": "Lake Superior", "lat": 47.7, "lon": -87.0, "continent": "North America"},
    {"name": "Lake Michigan", "lat": 44.0, "lon": -87.0, "continent": "North America"},
    {"name": "Lake Huron", "lat": 45.5, "lon": -83.3, "continent": "North America"},
    {"name": "Lake Winnipeg", "lat": 52.0, "lon": -97.0, "continent": "North America"},
    {"name": "Great Bear Lake", "lat": 65.9, "lon": -121.3, "continent": "North America"},
    {"name": "Great Slave Lake", "lat": 62.7, "lon": -114.4, "continent": "North America"},
    {"name": "Lake Okeechobee", "lat": 27.0, "lon": -81.0, "continent": "North America"},
    {"name": "Lake Titicaca", "lat": -15.8, "lon": -69.5, "continent": "South America"},
    {"name": "Lake Mar Chiquita", "lat": -30.0, "lon": -62.0, "continent": "South America"},
    {"name": "Lake Poopó", "lat": -19.0, "lon": -66.9, "continent": "South America"},
    {"name": "Lagoa dos Patos", "lat": -31.8, "lon": -52.2, "continent": "South America"},
    {"name": "Laguna Colorada", "lat": -22.3, "lon": -67.8, "continent": "South America"},
    {"name": "Laguna Blanca", "lat": -22.3, "lon": -67.9, "continent": "South America"},
    {"name": "Lake Geneva", "lat": 46.4, "lon": 6.6, "continent": "Europe"},
    {"name": "Lake Ladoga", "lat": 60.9, "lon": 31.5, "continent": "Europe"},
    {"name": "Lake Vänern", "lat": 58.6, "lon": 13.1, "continent": "Europe"},
    {"name": "Lake Garda", "lat": 45.6, "lon": 10.7, "continent": "Europe"},
    {"name": "Lake Constance", "lat": 47.6, "lon": 9.4, "continent": "Europe"},
    {"name": "Loch Ness", "lat": 57.3, "lon": -4.5, "continent": "Europe"},
    {"name": "Lake Sevan", "lat": 40.3, "lon": 45.3, "continent": "Europe"},
    {"name": "Qinghai Lake", "lat": 36.9, "lon": 100.0, "continent": "Asia"},
    {"name": "Lake Namtso", "lat": 30.5, "lon": 90.5, "continent": "Asia"},
    {"name": "Yamdrok Lake", "lat": 28.9, "lon": 90.5, "continent": "Asia"},
    {"name": "Issyk-Kul", "lat": 42.4, "lon": 77.3, "continent": "Asia"},
    {"name": "Lake Balkhash", "lat": 46.0, "lon": 74.7, "continent": "Asia"},
    {"name": "Aral Sea", "lat": 45.0, "lon": 59.0, "continent": "Asia"},
    {"name": "Lake Toba", "lat": 2.6, "lon": 98.8, "continent": "Asia"},
    {"name": "Tonlé Sap", "lat": 13.4, "lon": 104.0, "continent": "Asia"},
    {"name": "Caspian Sea", "lat": 46.0, "lon": 52.0, "continent": "Asia"},
    {"name": "Lake Baikal", "lat": 53.5, "lon": 108.2, "continent": "Asia"},
    {"name": "Lake Vostok", "lat": -77.3, "lon": 106.8, "continent": "Antarctica"},
    {"name": "Lake Untersee", "lat": -71.3, "lon": 13.5, "continent": "Antarctica"},
    {"name": "Lake Fryxell", "lat": -77.6, "lon": 163.1, "continent": "Antarctica"},
    {"name": "Lake Bonney", "lat": -77.7, "lon": 162.1, "continent": "Antarctica"},
    {"name": "Lake Hoare", "lat": -77.6, "lon": 162.9, "continent": "Antarctica"},
    {"name": "Lake Vanda", "lat": -77.5, "lon": 161.7, "continent": "Antarctica"},
    {"name": "Lake Whillans", "lat": -84.2, "lon": -153.6, "continent": "Antarctica"}
]



In [ ]:
# 获取影像集合函数
def fetch_image_collection(lake_name, lon, lat, start_date, end_date, sensor):
    point = ee.Geometry.Point([lon, lat])
    region = point.buffer(10000).bounds()

    if sensor == 'Sentinel-2':
        dataset = "COPERNICUS/S2_SR_HARMONIZED"
        bands = ['B4', 'B3', 'B2']
    elif sensor == 'Landsat-7':
        dataset = "LANDSAT/LE07/C02/T1_L2"
        bands = ['SR_B3', 'SR_B2', 'SR_B1']
    elif sensor == 'MODIS':
        dataset = "MODIS/061/MOD09GA"
        bands = ['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03']
    else:
        raise ValueError("Unsupported sensor")

    collection = (ee.ImageCollection(dataset)
                  .filterBounds(region)
                  .filterDate(start_date, end_date)
                  .select(bands))
    return collection, region

# 收集影像数量数据
records = []
for lake in lakes:
    name, lon, lat = lake['name'], lake['lon'], lake['lat']
    print(f"Processing: {name}")
    try:
        s2, _ = fetch_image_collection(name, lon, lat, "2015-01-01", "2024-06-01", "Sentinel-2")
        modis, _ = fetch_image_collection(name, lon, lat, "2000-01-01", "2024-06-01", "MODIS")
        l7, _ = fetch_image_collection(name, lon, lat, "1999-01-01", "2012-12-31", "Landsat-7")

        records.append({
            "Lake": name,
            "Continent": lake["continent"],
            "Sentinel-2": s2.size().getInfo(),
            "MODIS": modis.size().getInfo(),
            "Landsat-7": l7.size().getInfo()
        })
    except Exception as e:
        print(f"Error processing {name}: {e}")

df = pd.DataFrame(records)
print(df.head())



Processing: Lake Victoria
Processing: Lake Tanganyika
Processing: Lake Chad
Processing: Lake Tana
Processing: Lake Albert
Processing: Lake Malawi
Processing: Lake Turkana
Processing: Lake Eyasi
Processing: Lake Volta
Processing: Lake Mweru
Processing: Great Salt Lake
Processing: Lake Ontario
Processing: Lake Erie
Processing: Lake Superior
Processing: Lake Michigan
Processing: Lake Huron
Processing: Lake Winnipeg
Processing: Great Bear Lake
Processing: Great Slave Lake
Processing: Lake Okeechobee
Processing: Lake Titicaca
Processing: Lake Mar Chiquita
Processing: Lake Poopó
Processing: Lagoa dos Patos
Processing: Laguna Colorada
Processing: Laguna Blanca
Processing: Lake Geneva
Processing: Lake Ladoga
Processing: Lake Vänern
Processing: Lake Garda
Processing: Lake Constance
Processing: Loch Ness
Processing: Lake Sevan
Processing: Qinghai Lake
Processing: Lake Namtso
Processing: Yamdrok Lake
Processing: Issyk-Kul
Processing: Lake Balkhash
Processing: Aral Sea
Processing: Lake Toba
Proces

In [ ]:
# 控件定义
year_slider = widgets.IntSlider(
    value=2020,
    min=2015,
    max=2024,
    step=1,
    description='年份:',
    continuous_update=False
)

lake_dropdown = widgets.Dropdown(
    options=[lake["name"] for lake in lakes],
    description='湖泊:'
)

display(widgets.HBox([lake_dropdown, year_slider]))

# NDWI 计算函数
def calculate_ndwi(image):
    green = image.select("B3")
    nir = image.select("B8")
    ndwi = green.subtract(nir).divide(green.add(nir)).rename("NDWI")
    return image.addBands(ndwi)

# 主地图更新函数
def update_ndwi_map(lake_name, year):
    lake = next(l for l in lakes if l["name"] == lake_name)
    lon, lat = lake["lon"], lake["lat"]
    region = ee.Geometry.Point([lon, lat]).buffer(10000).bounds()
    start = f"{year}-01-01"
    end = f"{year}-12-31"

    collection = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                  .filterBounds(region)
                  .filterDate(start, end)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)))

    image = collection.median()
    ndwi_image = calculate_ndwi(image)

    vis_params = {"min": -1, "max": 1, "palette": ["brown", "white", "blue"]}

    m = geemap.Map()
    m.centerObject(region, 8)
    m.addLayer(ndwi_image.select("NDWI"), vis_params, f"{lake_name} NDWI {year}")
    m.addLayer(ee.FeatureCollection([ee.Feature(ee.Geometry.Point([lon, lat]))]),
               {'color': 'red'}, f"{lake_name}_位置")
    display(m)


In [ ]:
# 绑定控件与地图刷新
widgets.interact(update_ndwi_map, lake_name=lake_dropdown, year=year_slider)

interactive(children=(Dropdown(description='湖泊:', index=12, options=('Lake Victoria', 'Lake Tanganyika', 'Lake…

<function __main__.update_ndwi_map(lake_name, year)>